**Step 3. Sentiment Analysis with VADER**

Now we repeat the first chunks like in the steps before.After that we load the data, which was prepared in step 2, which will be used for the computation of sentiment.

In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
import nltk.corpus
from nltk.tokenize import RegexpTokenizer
import os
from dask import bag as db 
from dask import dataframe as dd

In [2]:
from dask.distributed import Client
client=Client(n_workers=8,)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 15.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60659,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.67 GiB
Comm: tcp://127.0.0.1:60698,Total threads: 1
Dashboard: http://127.0.0.1:60699/status,Memory: 1.96 GiB
Nanny: tcp://127.0.0.1:60664,


In [3]:
#loading the dataframe:

df = dd.read_parquet(r'C:\Users\artem\Desktop\Term_Paper_Sentiment_Crypto\lemmatization')

df.head(5)

,user_id,countStatus,user_screen_name,text_string_lem
tweet_id,,,,
1511650360317861894,1510316194418597897,8,JaineetKing,purplenftmuseum talkado nft whitelist live tal...
1511650360385052680,1452885627595149312,1,MohdNazriMohdN9,ameergiveaway hey guys ameer made new account ...
1511650360414584836,1487770961176719367,3,kborsalino00,yogetoth hello elefbody starting april 2022 ut...
1511650360628293635,1507030320847245312,2,Alikhan13319125,lacryptomonnai1 real utility development 1000x...
1511650360758263811,1171555019478507520,4,Imola_red888,locommunitynft fatima living life spots grabs ...


Downloading the vader sentiment lexicon:

In [ ]:
nltk.download('vader_lexicon')

Sentiment Intensity Analyzer:

Initializing an object of SentimentIntensityAnalyzer with name “analyzer”:

In [5]:
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

Computing polarity scores:

Use the polarity_scores method. The polarity score computes the overall sentiment by assigning negative, neutral and positive values to the text. On top of that, it computes the compound by normalizing these three scores. The compound varies between -1 and 1. The more negative the compound is, the more negative is the sentiment for the specific tweet (and vice versa).

In [6]:
df['polarity'] = df['text_string_lem'].apply(lambda x: analyzer.polarity_scores(x))

C:\Users\artem\anaconda3\lib\site-packages\dask\dataframe\core.py:3956: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text_string_lem', 'object'))

  warnings.warn(meta_warning(meta))


In [7]:
df.head(3)

,user_id,countStatus,user_screen_name,text_string_lem,polarity
tweet_id,,,,,
1511650360317861894,1510316194418597897,8,JaineetKing,purplenftmuseum talkado nft whitelist live tal...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1511650360385052680,1452885627595149312,1,MohdNazriMohdN9,ameergiveaway hey guys ameer made new account ...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp..."
1511650360414584836,1487770961176719367,3,kborsalino00,yogetoth hello elefbody starting april 2022 ut...,"{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compou..."


Here we see that the four sentiment variables are in one column. This needs to be restructured in the next steps:

In [8]:
df.dtypes

user_id              int64
countStatus          int64
user_screen_name    object
text_string_lem     object
polarity            object
dtype: object

In [9]:
#Transform data

# Change data structure
df1 = dd.concat(
    [df.drop('polarity', axis=1), 
     df['polarity'].apply(pd.Series, index = ['compound','neg','neu','pos'], meta={'compound': 'int64', 'neg': 'int64', 'neu': 'int64', 'pos': 'int64'})], axis=1)

C:\Users\artem\anaconda3\lib\site-packages\dask\dataframe\multi.py:1254: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [10]:
df1.head(5)

,user_id,countStatus,user_screen_name,text_string_lem,compound,neg,neu,pos
tweet_id,,,,,,,,
1511650360317861894,1510316194418597897,8,JaineetKing,purplenftmuseum talkado nft whitelist live tal...,0.0000,0.0,1.000,0.000
1511650360385052680,1452885627595149312,1,MohdNazriMohdN9,ameergiveaway hey guys ameer made new account ...,0.3400,0.0,0.854,0.146
1511650360414584836,1487770961176719367,3,kborsalino00,yogetoth hello elefbody starting april 2022 ut...,0.4404,0.0,0.860,0.140
1511650360628293635,1507030320847245312,2,Alikhan13319125,lacryptomonnai1 real utility development 1000x...,0.0000,0.0,1.000,0.000
1511650360758263811,1171555019478507520,4,Imola_red888,locommunitynft fatima living life spots grabs ...,0.3612,0.0,0.762,0.238


Now we see, that our table looks clean and the sentiment values are structured. Next step is to create a new column, called "sentiment", which looks at the compound and assigns a sentiment-category to the tweet:

In [11]:
# Create new variable with sentiment "neutral," "positive" and "negative"

df1['sentiment'] = df1['compound'].apply(lambda x: 'positive' if x >0 else 'neutral' if x==0 else 'negative')

C:\Users\artem\anaconda3\lib\site-packages\dask\dataframe\core.py:3956: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('compound', 'object'))

  warnings.warn(meta_warning(meta))


In [12]:
df1.head(5)

,user_id,countStatus,user_screen_name,text_string_lem,compound,neg,neu,pos,sentiment
tweet_id,,,,,,,,,
1511650360317861894,1510316194418597897,8,JaineetKing,purplenftmuseum talkado nft whitelist live tal...,0.0000,0.0,1.000,0.000,neutral
1511650360385052680,1452885627595149312,1,MohdNazriMohdN9,ameergiveaway hey guys ameer made new account ...,0.3400,0.0,0.854,0.146,positive
1511650360414584836,1487770961176719367,3,kborsalino00,yogetoth hello elefbody starting april 2022 ut...,0.4404,0.0,0.860,0.140,positive
1511650360628293635,1507030320847245312,2,Alikhan13319125,lacryptomonnai1 real utility development 1000x...,0.0000,0.0,1.000,0.000,neutral
1511650360758263811,1171555019478507520,4,Imola_red888,locommunitynft fatima living life spots grabs ...,0.3612,0.0,0.762,0.238,positive


In the following process we start looking into our data for analysis. At first we look into the most negative and the most positive posts to manually check if our algorithm accurately categorized the sentiment. The tables show, that our program performed well:

In [21]:
#creating a sorted dataframe
df_sorted = df1.sort_values('compound', axis=0, ascending=True)

The 10 most negative posts:

In [22]:
df_sorted.head(10)

,user_id,countStatus,user_screen_name,text_string_lem,compound,neg,neu,pos,sentiment
tweet_id,,,,,,,,,
1511732595532390403,1413805030189289472,978966,Cryptofalcon_,fuck bitcoin fuck bitcoin fuck bitcoin fuck bi...,-0.9967,0.760,0.240,0.000,negative
1512200003858673667,1142190723682488320,6524546,LairofD,takes based nobody disagrees anyways bring met...,-0.9945,0.672,0.297,0.031,negative
1511692370953138176,1365466861958168578,499427,aldigalaxi,scam project fuckers scam airdrop fuckers scam...,-0.9921,0.856,0.144,0.000,negative
1511692382974386189,1505346672800387073,499574,jo33004126,scam project fuckers scam airdrop fuckers scam...,-0.9921,0.856,0.144,0.000,negative
1512241862454628352,2183895764,7009424,VillarosaArts,ign wanda even main villain multiverse madness...,-0.9907,0.708,0.292,0.000,negative
1512295163372134409,1476806565948805122,7630546,Sketchy_jpg,common art stolen turned nft uses inefficient ...,-0.9903,0.717,0.246,0.037,negative
1512161490673016837,1478351783319777281,6073159,terrific646,bloomberglive crypto wilmerhale biden team plu...,-0.9902,0.633,0.260,0.107,negative
1511858215167959042,1431956223767814148,2467942,ourseronde,hell yeah hell yeah fuck hell yeah hell yeah f...,-0.9899,0.716,0.046,0.238,negative
1511849409943851028,1586904265,2363932,jamesmoylan77,need ban trading countries causing mass migrat...,-0.9878,0.632,0.368,0.000,negative


The 10 most positive posts:

In [23]:
df_sorted.tail(10)

,user_id,countStatus,user_screen_name,text_string_lem,compound,neg,neu,pos,sentiment
tweet_id,,,,,,,,,
1511670811559854082,1484537575515713537,242426,farhan_happy75,amazing project love concept super nice clean ...,0.9911,0.000,0.067,0.933,positive
1511789774318850076,85605967,1653855,HeathLegend,best direction gregor_jordan win best original...,0.9915,0.000,0.296,0.704,positive
1511780149259292677,1096503280740503552,1545814,neo_sans,sweetheart love sweetheart love sweetheart lov...,0.9918,0.045,0.103,0.851,positive
1511674781023694850,3279423091,289755,Cryptoboy_18,unfrgtnnft hustle_dao best dao hustle_dao best...,0.9928,0.000,0.333,0.667,positive
1512083272121020419,1511669363920687107,5140831,Mikasabigwin_13,done manifesting win win win win win win win w...,0.9936,0.045,0.132,0.823,positive
1511669847285207041,1438075887707066368,230936,ishida71,unfrgtnnft love hustle dao love hustle dao lov...,0.9940,0.000,0.332,0.668,positive
1511664619470741506,341219511,168619,Pharrellisgod,unfrgtnnft love hustle dao love hustle dao lov...,0.9940,0.000,0.332,0.668,positive
1511664450495197184,341219511,166597,Pharrellisgod,unfrgtnnft love hustle dao love hustle dao lov...,0.9940,0.000,0.332,0.668,positive
1511733294039191554,1511715629526573056,987281,LIL_DONALD_TRUM,jesus christ bible christianity xrp please ple...,0.9951,0.000,0.237,0.763,positive


While trying to save the dataset into parquet, we were faced with an issue that our numeric values (especially the floats) were lost during the conversion. In order to solve this problem we decided to convert all the numeric values into strings, which can be safely put into parquet files:

In [45]:
df1.dtypes
#creating the strings out of the compound column and others:
df1['compound'] = df1['compound'].astype(str)
df1['user_id'] = df1['user_id'].astype(str)
df1['neg'] = df1['neg'].astype(str)
df1['neu'] = df1['neu'].astype(str)
df1['pos'] = df1['compound'].astype(str)

user_id               int32
countStatus           int64
user_screen_name     object
text_string_lem      object
polarity             object
compound            float64
neg                 float64
neu                 float64
pos                 float64
dtype: object

In [46]:
df.head(5)

,user_id,countStatus,user_screen_name,text_string_lem,polarity,compound,neg,neu,pos
tweet_id,,,,,,,,,
1511650360317861894,-2147483648,8,JaineetKing,purplenftmuseum talkado nft whitelist live tal...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,1.000,0.000
1511650360385052680,-2147483648,1,MohdNazriMohdN9,ameergiveaway hey guys ameer made new account ...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp...",0.3400,0.0,0.854,0.146
1511650360414584836,-2147483648,3,kborsalino00,yogetoth hello elefbody starting april 2022 ut...,"{'neg': 0.0, 'neu': 0.86, 'pos': 0.14, 'compou...",0.4404,0.0,0.860,0.140
1511650360628293635,-2147483648,2,Alikhan13319125,lacryptomonnai1 real utility development 1000x...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,1.000,0.000
1511650360758263811,-2147483648,4,Imola_red888,locommunitynft fatima living life spots grabs ...,"{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'comp...",0.3612,0.0,0.762,0.238


In [39]:
#exporting the dataframe to parquet so in the following steps less data has to be sorted and computed:

df1.to_parquet(r'C:\Users\artem\Desktop\Term_Paper_Sentiment_Crypto\vader_sentiment_computed', compression='gzip')